In [18]:
import numpy as np
import panel as pn
import pandas as pd
import holoviews as hv
pn.extension('echarts')
from time import sleep
import shutup
shutup.please()
from random import choice
from os.path import join
import os

In [25]:
# dataset for the timeline graph
# dftest = pd.read_csv(join(os.pardir, 'data', 'china_cities_GDP.csv')) # locally
dftest = pd.read_csv('data/china_cities_GDP.csv') # remote

FileNotFoundError: [Errno 2] No such file or directory: 'data/china_cities_GDP.csv'

In [20]:
# timeline graph
def timeline(df,title='',subtitle='',xtitle=None,ytitle=''):
    yearslist = df.columns.tolist()[1:]
    xtitle = xtitle if xtitle else df.columns.tolist()[0]
    yearslist = df.columns.tolist()[1:]
    citylist = df.iloc[:,0].to_list()
    dfvalues = df.iloc[:,1:].values.tolist()
    options = []
    for year in yearslist:
        tempo = {"title" :{"text": f'{year} {title}'},
                "series": [
                {"data": df[year].to_list()}
                ]
                }
        options.append(tempo)
    echart = {
        'baseOption': {
            'timeline': {
                'axisType': 'category',
                'autoPlay': True,
                'playInterval': 1000,
                'data': yearslist
            },
            'title': {
                'subtext': f'{subtitle}',
            },
            'tooltip': {},
            'legend':{
                'left': 'right',
                },
            'calculable' : True,
            'grid': {
                'top': 80,
                'bottom': 100
            },
            'xAxis': [
                {
                    'type':'category',
                    'axisLabel':{'interval':0, 'rotate': 45},
                    'data': citylist,
                    'splitLine': {'show': False},
                    'name': f'{xtitle}',
                }
            ],
            'yAxis': [
                {
                    'type': 'value',
                    'name': f'{ytitle}',
                    'max': 'dataMax',
                }
            ],
            'series': [
                {'name': 'GDP',
                 'type': 'bar',
                 # 'colorBy': 'data', 
                 # 'realtimeSort': True, # comment for normal chat
                },
                ]
        },
            'options': options,

        };
    
    return echart

timeline_dict = timeline(dftest,title='GDP',subtitle='hell yeah baby',xtitle='City',ytitle='$')

timeline_pane = pn.pane.ECharts(timeline_dict, height=600, width=1000, theme='default')
timeline_pane.servable(title='Timeline App')

ECharts(dict, height=600, sizing_mode='fixed', width=1000)

In [21]:
# dataset for the line race and bar race graphs

df = pd.read_json('data/life-expectancy-table.json')
df.columns = ['Income','Life Expectancy','Population','Country','Year']
df.dropna(inplace=True)
df_flags = pd.read_json('data/flags.json')
df_flags.columns = ["code","emoji","unicode","name","title","dialCode"]
df = df.merge(df_flags,how='inner',left_on='Country',right_on='name')
df = df[['Income','Life Expectancy','Population','Country','Year','emoji']]
df.sort_values(['Country', 'Year'],inplace=True)
hex_codes = '0123456789abcdef'
colors = []
items = df['Country'].unique().tolist()
for i in range(len(items)):
    combination = '#' + ''.join((choice(hex_codes), choice(hex_codes), choice(hex_codes), choice(hex_codes), choice(hex_codes), choice(hex_codes)))
    colors.append(combination)

color_dict = {i:j for i,j in zip(items,colors)}
df['color'] = df['Country'].map(color_dict)
small_df = df[df['Country'].isin(['Finland','France','Germany','Iceland','Norway','Poland','Russia','Australia','Japan','Turkey','Canada'])]

In [22]:
#line race graph
x= 'Year'
y= 'Income'
filter_col= 'Country'
lrdatasetWithFilters = []
lrseriesList = []
lritems = small_df[filter_col].unique().tolist()
lrchunks = sorted(small_df[x].unique().tolist())
for item in lritems:
    lrfiltereddf = small_df[(df[filter_col] == item) & (small_df[x] == lrchunks[0])]
    datasetId = 'dataset_' + item
    lrdatasetWithFilters.append({
      'id': datasetId,
      'source': [list(lrfiltereddf.columns)] + lrfiltereddf.values.tolist(),
    })
    lrseriesList.append({
      'type': 'line',
      'datasetId': datasetId,
      'showSymbol': False,
      'name': item,
      'endLabel': {
        'show': True,
        'formatter': '{a}: {@%s}' % y
      },
      'labelLayout': {
        'moveOverlap': 'shiftY'
      },
      'emphasis': {
        'focus': 'series'
      },
      'encode': {
        'x': x,
        'y': y,
        'label': [filter_col, y],
        'tooltip': [y],
      }
    });
lroption = {
'animationDuration': 10000,
'dataset': [*lrdatasetWithFilters],
'title': {
'text': 'Line Race With Echarts and Panel',
    'textStyle': {
        'color': '#FFFFFF',
        },
},
'tooltip': {
  'order': 'valueDesc',
  'trigger': 'axis'
},
'xAxis': {
  'type': 'category',
  'nameLocation': 'middle',
},
'yAxis': {
  'name': y,
},
'grid': {
  'right': 140
},
'series': lrseriesList,
};

linerace_pane = pn.pane.ECharts(lroption, height=400, width=1000, theme='light')
lrbutton = pn.widgets.Button(name='► Play', width=60, align='end')

def linerace_update_graph():
    for i, chunck in enumerate(lrchunks[:-1]):
        lrdatasetWithFilters=[]
        for item in lritems:
            lrfiltereddf = small_df[(small_df[filter_col] == item) & (small_df[x].isin(lrchunks[:i+1]))]
            datasetId = 'dataset_' + item
            lrdatasetWithFilters.append({
              'id': datasetId,
              'source': [list(lrfiltereddf.columns)] + lrfiltereddf.values.tolist(),
            })
        linerace_pane.object['dataset'] = [*lrdatasetWithFilters]
        linerace_pane.param.trigger('object')
        sleep(0.25)


def linerace_animate(event):
    if lrbutton.name == '► Play':
        lrbutton.button_type = 'primary'
        lrbutton.name = 'Racing'
        linerace_update_graph()
        lrbutton.button_type = 'success'
        lrbutton.name = 'Completed'
    else:
        lrbutton.button_type = 'default'
        lrbutton.name = '► Play'

lrbutton.on_click(linerace_animate)


linerace_layout = pn.Column(linerace_pane, lrbutton,sizing_mode='stretch_both')
linerace_layout.servable(title='Line Racing App')

Column(sizing_mode='stretch_both')
    [0] ECharts(dict, height=400, sizing_mode='fixed', theme='light', width=1000)
    [1] Button(align='end', name='► Play', width=60)

In [23]:
# bar race graph
colors = ['#ECEA98',' #F93822','#9BB9D0','#507D2A','#E97451','#e3e3e3','#EEC61F','#A80B44','#7492B9', '#E1A95F','#826644','#fdf4ee']
x= 'Year'
y= 'Income'
filter_col= 'Country'
brdatasetWithFilters = []
brseriesList = []
chunks = sorted(small_df[x].unique().tolist())
items = small_df[filter_col].unique().tolist()
filtereddf = small_df[(small_df[x] == chunks[0])]
datasetId = 'dataset_' + str(chunks[0])
brdatasetWithFilters.append({
  'id': datasetId,
  'source': [list(filtereddf.columns)] + filtereddf.values.tolist(),
})
brseriesList.append({
  'realtimeSort': True,
  'type': 'bar',
  'colorBy': 'data',
  'datasetId': datasetId,
  'showSymbol': False,
  'name': chunks[0],
  # 'universalTransition': True,
  'encode': {
    'x': y,
    'y': filter_col,
  },
    'label': {
          'show': True,
          'position': 'right',
          'valueAnimation': True,
          'precision': 0,
        },  
 },
)
broption = {
'animationDuration': 10000,
'dataset': [*brdatasetWithFilters],
'title': {
'text': 'Bar Race With Echarts and Panel',
    'textStyle': {
        'color': '#FFFFFF',
        },
},
'tooltip': {
'order': 'valueDesc',
'trigger': 'axis'
},
'xAxis': {
'type': 'value',
'nameLocation': 'middle',
'max': 'dataMax',
},
'yAxis': {
'type': 'category',
'inverse': True,
'max': 10,
'axisLabel': {
        'show': True,
        'fontSize': 14,
        'formatter': '{value}',
      },
'animationDuration': 300,
'animationDurationUpdate': 300
},
'grid': {
      'top': 30,
      'bottom': 30,
      'left': 150,
      'right': 80
    },
'animationDuration': 0,
'animationDurationUpdate': 2000,
'animationEasing': 'linear',
'animationEasingUpdate': 'linear',
'graphic': {
  'elements': [
    {
      'type': 'text',
      'right': 160,
      'bottom': 60,
      'style': {
        'text': chunks[0],
        'font': 'bolder 80px monospace',
        'fill': 'rgba(100, 100, 100, 0.25)'
      },
      'z': 100
    }
  ]
},
'series': brseriesList,
'color': colors,
};
barrace_pane = pn.pane.ECharts(broption, height=400, width=1000, theme='light')
brbutton = pn.widgets.Button(name='► Play', width=60, align='end')

def barrace_animate(event):
    if brbutton.name == '► Play':
        brbutton.button_type = 'primary'
        brbutton.name = 'Racing'
        barrace_update_graph()
        brbutton.button_type = 'success'
        brbutton.name = 'Completed'
    else:
        brbutton.button_type = 'default'
        brbutton.name = '► Play'

brbutton.on_click(barrace_animate)



def barrace_update_graph():
    for chunk in chunks:
        brdatasetWithFilters=[]
        brseriesList = []
        filtereddf = small_df[(small_df[x] == chunk)]
        datasetId = 'dataset_' + str(chunk)
        brdatasetWithFilters.append({
          'id': datasetId,
          'source': [list(filtereddf.columns)] + filtereddf.values.tolist(),
        })
        brseriesList.append({
        'realtimeSort': True,
        'type': 'bar',
        'colorBy': 'data',
        'datasetId': datasetId,
        'showSymbol': False,
        'name': chunk,
        'encode': {
        'x': y,
        'y': filter_col,
        },
        'label': {
          'show': True,
          'precision': 0,
          'position': 'right',
          'valueAnimation': True,
          'fontFamily': 'monospace'
        },
        })
        broption = {
        'animationDuration': 10000,
        'dataset': [*brdatasetWithFilters],
        'title': {
        'text': 'Bar Race With Echarts and Panel',
        'textStyle': {
        'color': '#FFFFFF',
        },
        },
        'tooltip': {
        'order': 'valueDesc',
        'trigger': 'axis'
        },
        'xAxis': {
        'type': 'value',
        'nameLocation': 'middle',
        'max': 'dataMax',
        },
        'yAxis': {
        'type': 'category',
        'inverse': True,
        'max': 10,
        'axisLabel': {
        'show': True,
        'fontSize': 14,
        'formatter': '{value}',
        },
        'animationDuration': 300,
        'animationDurationUpdate': 300,
        },
        'grid': {
      'top': 30,
      'bottom': 30,
      'left': 150,
      'right': 80
        },
        'animationDuration': 0,
        'animationDurationUpdate': 2000,
        'animationEasing': 'linear',
        'animationEasingUpdate': 'linear',
        'graphic': {
        'elements': [
        {
          'type': 'text',
          'right': 160,
          'bottom': 60,
          'style': {
            'text': chunk,
            'font': 'bolder 80px monospace',
            'fill': 'rgba(100, 100, 100, 0.25)'
          },
          'z': 100
        }
        ]
        },
        'series': brseriesList,
        'color': colors,
        };
        barrace_pane.object = broption
        barrace_pane.param.trigger('object')
        sleep(0.25)        

barrace_layout = pn.Column(barrace_pane, brbutton,sizing_mode='stretch_both')
barrace_layout.servable(title='Bar Racing App')

Column(sizing_mode='stretch_both')
    [0] ECharts(dict, height=400, sizing_mode='fixed', theme='light', width=1000)
    [1] Button(align='end', name='► Play', width=60)

In [24]:
# pn.serve({
#     'markdown': markdown_app,
#     'json': json_app,
#     'layout': layout_app,
# },threaded=True)

# slider = pn.widgets.IntSlider(start=0, end=10)

# def slideshow(index):
#     url = f"https://picsum.photos/800/300?image={index}"
#     return pn.pane.JPG(url)

# output = pn.bind(slideshow, slider)

# app = pn.Column(slider, output)